#Assignment 3 - Fine Tuning a Language Model
##Named Entity Recognition and Gradio UI
Using Huggingface example - https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/token_classification.ipynb

In [1]:
! pip install datasets transformers seqeval

     |████████████████████████████████| 270 kB 5.3 MB/s 
     |████████████████████████████████| 2.9 MB 31.1 MB/s 
     |████████████████████████████████| 43 kB 1.7 MB/s 
     |████████████████████████████████| 1.3 MB 33.4 MB/s 
     |████████████████████████████████| 243 kB 43.2 MB/s 
     |████████████████████████████████| 56 kB 4.6 MB/s 
     |████████████████████████████████| 125 kB 44.5 MB/s 
     |████████████████████████████████| 3.3 MB 37.2 MB/s 
     |████████████████████████████████| 636 kB 29.9 MB/s 
     |████████████████████████████████| 895 kB 37.0 MB/s 
     |████████████████████████████████| 271 kB 47.5 MB/s 
     |████████████████████████████████| 160 kB 47.1 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=d146d1b304bc50b45746d2312e25fc6d8ddc6987d842060ed6f15d003299ad97
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval
  Attempting uninstall:

In [2]:
# Do all the imports

import numpy as np
from datasets import load_dataset, load_metric
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML
import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification


print(transformers.__version__)

4.11.3


In [3]:
task = "ner" 
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

In [4]:
datasets = load_dataset('conll2003')
datasets

Downloading:   0%|          | 0.00/2.60k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/650k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/163k [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/40e7cb6bcc374f7c349c83acd1e9352a4f09474eb691f64f364ee62eb65d0ca6. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [5]:
datasets['train'][100]

{'chunk_tags': [11, 21, 22, 15, 11, 12, 13, 11, 0],
 'id': '100',
 'ner_tags': [1, 0, 0, 0, 0, 0, 0, 0, 0],
 'pos_tags': [21, 42, 39, 33, 29, 21, 15, 21, 7],
 'tokens': ['Rabinovich',
  'is',
  'winding',
  'up',
  'his',
  'term',
  'as',
  'ambassador',
  '.']}

In [6]:
datasets["train"].features[f"ner_tags"]

Sequence(feature=ClassLabel(num_classes=9, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], names_file=None, id=None), length=-1, id=None)

In [7]:
# Create the labels
label_list = datasets["train"].features[f"{task}_tags"].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

## Preprocess the Data

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [9]:
label_all_tokens = True

In [10]:
# Function to tokenize and align the labels

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [11]:
tokenize_and_align_labels(datasets['train'][5:10])

{'input_ids': [[101, 1000, 2057, 2079, 1050, 1005, 1056, 2490, 2151, 2107, 12832, 2138, 2057, 2079, 1050, 1005, 1056, 2156, 2151, 5286, 2005, 2009, 1010, 1000, 1996, 3222, 1005, 1055, 2708, 14056, 24794, 2271, 3158, 4315, 14674, 2409, 1037, 2739, 27918, 1012, 102], [101, 2002, 2056, 2582, 4045, 2817, 2001, 3223, 1998, 2065, 2009, 2001, 2179, 2008, 2895, 2001, 2734, 2009, 2323, 2022, 2579, 2011, 1996, 2647, 2586, 1012, 102], [101, 2002, 2056, 1037, 6378, 2197, 3204, 2011, 7327, 3888, 5849, 8965, 27424, 2818, 3917, 2000, 7221, 8351, 14332, 1010, 11867, 24129, 2015, 1998, 16492, 24551, 2013, 1996, 2529, 1998, 4111, 2833, 8859, 2001, 1037, 3811, 3563, 1998, 3653, 3540, 13700, 5649, 2693, 2000, 4047, 2529, 2740, 1012, 102], [101, 27424, 2818, 3917, 3818, 7327, 1011, 2898, 5761, 2044, 4311, 2013, 3725, 1998, 2605, 2008, 2104, 5911, 3785, 8351, 2071, 3206, 8945, 20534, 11867, 5063, 22631, 4372, 3401, 21890, 4135, 20166, 1006, 18667, 2063, 1007, 1011, 1011, 5506, 11190, 4295, 1012, 102], [101,

In [12]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

## Fine Tuning

In [13]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN t

In [14]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [15]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [16]:
metric = load_metric("seqeval")

Downloading:   0%|          | 0.00/2.48k [00:00<?, ?B/s]

In [17]:
# Function to compute metrics

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [18]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [19]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: tokens, id, pos_tags, chunk_tags, ner_tags.
***** Running training *****
  Num examples = 14041
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2634


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.244500,0.068920,0.913183,0.923705,0.918414,0.981238
2,0.051500,0.060173,0.923272,0.936906,0.930039,0.983558
3,0.031100,0.060027,0.927663,0.938248,0.932925,0.984272


Saving model checkpoint to distilbert-base-uncased-finetuned-ner/checkpoint-500
Configuration saved in distilbert-base-uncased-finetuned-ner/checkpoint-500/config.json
Model weights saved in distilbert-base-uncased-finetuned-ner/checkpoint-500/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-ner/checkpoint-500/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-ner/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: tokens, id, pos_tags, chunk_tags, ner_tags.
***** Running Evaluation *****
  Num examples = 3250
  Batch size = 16
Saving model checkpoint to distilbert-base-uncased-finetuned-ner/checkpoint-1000
Configuration saved in distilbert-base-uncased-finetuned-ner/checkpoint-1000/config.json
Model weights saved in distilbert-base-uncased-finetuned-ner/checkpoint-1000/pytorch_model.

TrainOutput(global_step=2634, training_loss=0.08670890865311445, metrics={'train_runtime': 512.3824, 'train_samples_per_second': 82.21, 'train_steps_per_second': 5.141, 'total_flos': 509926772226690.0, 'train_loss': 0.08670890865311445, 'epoch': 3.0})

## Gradio UI

In [20]:
! pip install gradio

     |████████████████████████████████| 3.4 MB 5.2 MB/s 
     |████████████████████████████████| 1.9 MB 34.0 MB/s 
     |████████████████████████████████| 206 kB 47.1 MB/s 
     |████████████████████████████████| 961 kB 26.9 MB/s 
     |████████████████████████████████| 3.5 MB 38.3 MB/s 
     |████████████████████████████████| 63 kB 2.4 MB/s 
  Created wheel for ffmpy: filename=ffmpy-0.3.0-py3-none-any.whl size=4710 sha256=01f81cc7ea62032ac1cf22182b5c3fbe55c410e683ed85ebac25298e612fbc3c
  Stored in directory: /root/.cache/pip/wheels/13/e4/6c/e8059816e86796a597c6e6b0d4c880630f51a1fcfa0befd5e6
  Created wheel for flask-cachebuster: filename=Flask_CacheBuster-1.0.0-py3-none-any.whl size=3371 sha256=d95b38f5f377ff534734ea1e6a04fcca985fa5e4c3e17106c2994a69e4eaae4d
  Stored in directory: /root/.cache/pip/wheels/28/c0/c4/44687421dab41455be93112bd1b0dee1f3c5a9aa27bee63708
Successfully built ffmpy flask-cachebuster


In [21]:
import gradio as gr

In [28]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)


loading configuration file https://huggingface.co/dslim/bert-base-NER/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a5ff16a1d557b5ad480f50b1d454448475c644d08df9ce8fccabea7745bebd9f.a61836f2236a3ff1a0827544e2d7c512cbb8cd26ed7b32d643526bebb5d7f92e
Model config BertConfig {
  "_num_labels": 9,
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-MISC",
    "2": "I-MISC",
    "3": "B-PER",
    "4": "I-PER",
    "5": "B-ORG",
    "6": "I-ORG",
    "7": "B-LOC",
    "8": "I-LOC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-LOC": 7,
    "B-MISC": 1,
    "B-ORG": 5,
    "B-PER": 3,
    "I-LOC": 8,
    "I-MISC": 2,
    "I-ORG": 6,
    "I-PER": 4,
    "O": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_typ

In [35]:
ner_results = nlp("this is a sentence and I live in San Francisco")
print(ner_results)

[{'entity': 'B-LOC', 'score': 0.99948984, 'index': 9, 'word': 'San', 'start': 33, 'end': 36}, {'entity': 'I-LOC', 'score': 0.99938995, 'index': 10, 'word': 'Francisco', 'start': 37, 'end': 46}]


In [32]:
def ner_analysis(text):
    text = text.encode("ascii", errors="ignore").decode(
        "ascii"
    )  
    results = nlp(text)
    return results

In [39]:
gradio_ui = gr.Interface(
    fn=ner_analysis,
    title="NER",
    description="Name Entity Recognition",
    inputs=gr.inputs.Textbox(lines=10, label="Paste some text here"),
    outputs=[
        gr.outputs.Textbox(label="Name Entity REcognition")
    ],
).launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
This share link will expire in 72 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted
Running on External URL: https://47141.gradio.app
Interface loading below...
